In [ ]:
# pip install uszipcode

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
import requests
import json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.listdir('drive/MyDrive/Freelance/001')

['Bancos.ipynb',
 'Datos_Bancos.xlsx',
 'Datos_Bancos_Lat_Lon.xlsx',
 'Datos_Bancos2006.xlsx',
 'Datos_Bancos2010.xlsx']

In [ ]:
path = 'drive/MyDrive/Freelance/001'
filename = 'Datos_Bancos.xlsx'
filepath = os.path.join(path, filename)
filepath

'drive/MyDrive/Freelance/001/Datos_Bancos.xlsx'

In [ ]:
xl = pd.ExcelFile(filepath)
xl.sheet_names

['2005', '2006', '2007', '2008', '2009', '2010']

In [ ]:
df2005 = pd.read_excel(filepath, sheet_name='2005')
df2006 = pd.read_excel(filepath, sheet_name='2006')
df2007 = pd.read_excel(filepath, sheet_name='2007')
df2008 = pd.read_excel(filepath, sheet_name='2008')
df2009 = pd.read_excel(filepath, sheet_name='2009')
df2010 = pd.read_excel(filepath, sheet_name='2010')
dfs = [df2005, df2006, df2007, df2008, df2009, df2010]

In [ ]:
for df in dfs:
  print(list(df.columns))

['IDRSSD', 'FDIC Certificate Number', 'Primary ABA Routing Number', 'Financial Institution Name', 'Financial Institution Address', 'Financial Institution City', 'Financial Institution State', 'Financial Institution Zip Code', 'Financial Institution Filing Type', 'Provisions', 'Asset Share', 'Failed Loans', 'Repite', 'Z_score', 'IL45', 'Disponible los 12 anios', 'Unnamed: 16']
['IDRSSD', 'FDIC Certificate Number', 'Primary ABA Routing Number', 'Financial Institution Name', 'Financial Institution Address', 'Financial Institution City', 'Financial Institution State', 'Financial Institution Zip Code', 'Financial Institution Filing Type', 'Provisions', 'Asset Share', 'Failed Loans', 'Repite', 'Z_score', 'IL45', 'Disponible los 12 anios', 'Unnamed: 16']
['IDRSSD', 'FDIC Certificate Number', 'Primary ABA Routing Number', 'Financial Institution Name', 'Financial Institution Address', 'Financial Institution City', 'Financial Institution State', 'Financial Institution Zip Code', 'Financial Insti

In [ ]:
cols = list(df.columns)
cols

['IDRSSD',
 'FDIC Certificate Number',
 'Primary ABA Routing Number',
 'Financial Institution Name',
 'Financial Institution Address',
 'Financial Institution City',
 'Financial Institution State',
 'Financial Institution Zip Code',
 'Financial Institution Filing Type',
 'Provisions',
 'Asset Share',
 'Failed Loans',
 'Repite',
 'Z_score',
 'IL45',
 'Disponible los 12 anios',
 'Unnamed: 16']

In [ ]:
cols_dir = ['Financial Institution Address', 'Financial Institution City', 'Financial Institution State']

In [ ]:
for df in dfs:
  print(df[cols_dir].isna().sum())

Financial Institution Address    0
Financial Institution City       0
Financial Institution State      0
dtype: int64
Financial Institution Address    0
Financial Institution City       0
Financial Institution State      0
dtype: int64
Financial Institution Address    0
Financial Institution City       0
Financial Institution State      0
dtype: int64
Financial Institution Address    0
Financial Institution City       0
Financial Institution State      0
dtype: int64
Financial Institution Address    0
Financial Institution City       0
Financial Institution State      0
dtype: int64
Financial Institution Address    0
Financial Institution City       0
Financial Institution State      0
dtype: int64


No hay datos nan en las direcciones

In [ ]:
n_rows_tot = 0
for df in dfs:
  print(f"Datos unicos: {len(df['IDRSSD'].unique())}")
  print(f"nan: {df['IDRSSD'].isna().sum()}")
  print(f"Filas: {len(df)}\n")
  n_rows_tot += len(df)
print(f"Filas totales: {n_rows_tot}")

Datos unicos: 7730
nan: 0
Filas: 7730

Datos unicos: 7611
nan: 0
Filas: 7611

Datos unicos: 7493
nan: 0
Filas: 7493

Datos unicos: 7274
nan: 0
Filas: 7274

Datos unicos: 7019
nan: 0
Filas: 7019

Datos unicos: 6694
nan: 0
Filas: 6694

Filas totales: 43821


Conclusión: hay que guiarse por el IDRSSD

In [ ]:
app = Nominatim(user_agent="tutorial")

In [ ]:
# Nominatim. Gratis

# row = 0
# for df in dfs:
#   for id in df['IDRSSD'].values:
#     mask = df['IDRSSD'] == id
#     FIA, FIC, State = df.loc[mask, cols_dir].values[0]
#     dir = f"{FIA.strip()}, {FIC}, {State}"
#     try:
#       location = app.geocode(dir).raw
#       df.loc[id, 'Latitude'] = location['lat']
#       df.loc[id, 'Longitude'] = location['lon']
#       print(dir)
#     except:
#       df.loc[id, 'Latitude'] = np.nan
#       df.loc[id, 'Longitude'] = np.nan
#     row += 1
#     print(f"{row/n_rows_tot*100:.2}%")

In [ ]:
len(dfs), len(dfs[:2]), len(dfs[2:])

(6, 2, 4)

In [ ]:
# GCP Geocoding API. 5 usd por cada 1000 peticiones
google_maps_api_url = "https://maps.googleapis.com/maps/api/geocode/json?"
api_key = YourApiKey

row = 0
for year, df in enumerate(dfs): # Cada dataframe (cada año)

  if year == 0: # Saltarse el año 2005
    continue

  for id in df['IDRSSD'].values: # Para cada IDRSSD
    mask = df['IDRSSD'] == id
    FIA, FIC, State = df.loc[mask, cols_dir].values[0]
    dir = f"{FIA.strip()}, {FIC}, {State}"
    params = {
    'key': api_key,
    'address': dir
    }
    print(dir) # Mostrar direccion actual
    response = requests.get(google_maps_api_url, params).json()
    if response["status"] == "OK":
      geom = response["results"][0]["geometry"]
      try:
        lat = geom['location']['lat']
        lng = geom['location']['lng']
        print("Bien") # Funciona
      except:
        lat = np.nan
        lng = np.nan
        print("No está en GCP") # No la encuentra
    else:
      lat = None
      lng = None
      print(f'Falla en respuesta: {response["status"]}') # No se conecta
    df.loc[mask, 'Latitude'] = lat
    df.loc[mask, 'Longitude'] = lng
    print(lat, lng)
    row += 1
    print(f"{row/n_rows_tot*100:.2f}%\n")
  if year == 0:
    year = 2005
  elif year == 1:
    year = 2006
  elif year == 2:
    year = 2007
  elif year == 3:
    year = 2008
  elif year == 4:
    year = 2009
  elif year == 5:
    year = 2010
  df_mod_path = os.path.join(path, "Datos_Bancos" + str(year) + ".xlsx")
  print(df_mod_path)
  print(40*"*")
  df.to_excel(df_mod_path, index=False)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
38.08529780000001 -90.0976301
80.08%

1005 CONGRESS AVENUE SUITE 100, AUSTIN, TX
Bien
30.2720098 -97.7408639
80.08%

115 WEST MARKET STREET, RED BUD, IL
Bien
38.2118679 -89.9943428
80.08%

187 148TH STREET, MANSFIELD, SD
Bien
45.2426189 -98.562812
80.09%

201 VALLEY, RUSHVILLE, MO
Bien
39.5866676 -95.0274002
80.09%

1514 MAIN STREET, SCOTT CITY, MO
Bien
37.2164352 -89.5265699
80.09%

201 WEST MAPLE, SLATER, MO
Bien
39.2192771 -93.06300429999999
80.09%

1525 EAST 53RD STREET, CHICAGO, IL
Bien
41.79940759999999 -87.5879867
80.10%

5230 SOUTH MAIN STREET, EMINENCE, KY
Bien
38.3691295 -85.1801336
80.10%

203 W COMANCHE AVENUE, SHABBONA, IL
Bien
41.7677666 -88.8738899
80.10%

113 MAIN STREET, KIT CARSON, CO
Bien
38.7632805 -102.7917526
80.10%

MAIN ST., BRICELYN, MN
Bien
43.555997 -93.812641
80.11%

880 SAN ANTONIO AVENUE, MANY, LA
Bien
31.5681268 -93.4828454
80.11%

400 MAIN STREET, ROCKWELL CITY, IA
Bien
42.3960283 -94.633

In [ ]:
# Cargar
df = pd.read_csv(df_mod_path, index_col=0)

In [ ]:
df["Latitude"].isna().sum()

2505

In [ ]:
nan_mask1 = df["Latitude"].isna()
df_nan1 = df.loc[nan_mask1].copy()

In [ ]:
dir_cols

['Financial Institution Address',
 'Financial Institution City',
 'Financial Institution State',
 'Financial Institution Zip Code']

In [ ]:
banco = df_nan1.iloc[0]
banco

FDIC Certificate Number                                                 91005
Primary ABA Routing Number                                          307087713
Financial Institution Name           5 STAR BANK (A COLORADO INDUSTRIAL BANK)
Financial Institution Address              PETERSON AIRFORCE BASE, BLDG 1485 
Financial Institution City                                   COLORADO SPRINGS
Financial Institution State                                                CO
Financial Institution Zip Code                                          80914
Financial Institution Filing Type                                          41
Provisions                                                           0.010991
Asset Share                                                          0.000014
Failed Loans                                                              0.0
Repite                                                                     12
Z_score                                                         

In [ ]:
# id = banco.index
# FIA, FIC, State = banco[["Financial Institution Address", "Financial Institution City", "Financial Institution State"]]
# zip = banco["Financial Institution Zip Code"]
# dir = f"{FIA.strip()}, {FIC}. {State}"

# params = {
#     'key': api_key,
#     'address': dir
# }

# response = requests.get(url, params).json()

# geom = response["results"][0]["geometry"]
# geom['location']['lat']
# geom['location']['lng']

-104.6995991

In [ ]:
# df_copy = df.copy()
df = df_copy.copy()

In [ ]:
len(df.index)

7730

In [ ]:
# # consola de google cloud. 5 usd por cada 1000 peticiones

# i = 2944
for id in df[nan_mask2].index:
  FIA, FIC, State = df[nan_mask2].loc[id][["Financial Institution Address", "Financial Institution City", "Financial Institution State"]]
  dir = f"{FIA.strip()}, {FIC}, {State}"
  params = {
    'key': api_key,
    'address': dir
  }
  response = requests.get(url, params).json()
  if response["status"] == "OK":
    geom = response["results"][0]["geometry"]
    try:
      lat = geom['location']['lat']
      lng = geom['location']['lng']
      df.loc[id, 'Latitude_GCP'] = lat
      df.loc[id, 'Longitude_GCP'] = lng
    except:
      lat = np.nan
      lng = np.nan
      df.loc[id, 'Latitude_GCP'] = lat
      df.loc[id, 'Longitude_GCP'] = lng
  else:
      lat = None
      lng = None
      df.loc[id, 'Latitude_GCP'] = lat
      df.loc[id, 'Longitude_GCP'] = lng

  i += 1
  print(i, dir)
  print(lat, lng)

Se han truncado las últimas 5000 líneas del flujo de salida.
5231 379 MAIN STREET, UNADILLA, NE
40.6816897 -96.2723125
5232 295 CENTER AVENUE, ASHTON, NE
41.2481306 -98.7943574
5233 14 LA BARRE STREET, GIBBON, NE
40.7487159 -98.8457086
5234 423 GRAND AVENUE, RAVENNA, NE
41.0270756 -98.91234159999999
5235 2050 BLUFF STREET, FULTON, MO
38.8693276 -91.9432535
5236 101 SOUTH MAIN STREET, CHARLESTON, MO
36.9093828 -89.35103840000001
5237 901 BUSINESS HIGHWAY 61 NORTH, BOWLING GREEN, MO
39.3557764 -91.1989058
5238 600 MAIN STREET, CABOOL, MO
37.1219655 -92.1010556
5239 18255 HWY 45 NORTH, WESTON, MO
39.4227829 -94.898955
5240 122 EAST MAIN STREET, HINCKLEY, MN
46.015 -92.9388889
5241 105 MAIN STREET, ELKTON, MN
43.663354 -92.708739
5242 611 ROSE DR., BIG LAKE, MN
45.3333582 -93.73934179999999
5243 318 1ST STREET NORTH, DARWIN, MN
45.0801824 -94.3066857
5244 250 3RD ST., TRACY, MN
44.2337972 -95.6184847
5245 11 N. CENTRAL AVENUE, KENSINGTON, MN
45.7770351 -95.69521309999999
5246 200 EAST FIRS

In [ ]:
nan_mask2 = df['Latitude_GCP'].isna()
example = df[nan_mask2].iloc[0]
example

FDIC Certificate Number                                   13130
Primary ABA Routing Number                             74909988
Financial Institution Name           FRIENDSHIP STATE BANK, THE
Financial Institution Address            5908 EAST MAIN STREET 
Financial Institution City                           FRIENDSHIP
Financial Institution State                                  IN
Financial Institution Zip Code                            47021
Financial Institution Filing Type                            41
Provisions                                             0.001781
Asset Share                                            0.000013
Failed Loans                                           0.004674
Repite                                                       12
Z_score                                               45.559483
IL45                                                   0.535802
Disponible los 12 anios                                       1
Unnamed: 16                             

In [ ]:
df[['Latitude', 'Longitude']]

,Latitude,Longitude
IDRSSD,,
1397471,34.750332,-92.278560
455253,NaN,NaN
896856,39.376352,-104.859526
522959,38.514789,-107.917269
734613,40.343500,-77.304215
...,...,...
2218425,41.018096,-92.413407
765774,28.296980,-97.275514
449878,37.334692,-79.524717


In [ ]:
mask3 = df['Latitude_GCP'].isna()
mask4 = df['Longitude_GCP'].isna()

In [ ]:
df[mask3]

,FDIC Certificate Number,Primary ABA Routing Number,Financial Institution Name,Financial Institution Address,Financial Institution City,Financial Institution State,Financial Institution Zip Code,Financial Institution Filing Type,Provisions,Asset Share,Failed Loans,Repite,Z_score,IL45,Disponible los 12 anios,Unnamed: 16,Latitude,Longitude,Latitude_GCP,Longitude_GCP
IDRSSD,,,,,,,,,,,,,,,,,,,,
427241,13130,74909988,"FRIENDSHIP STATE BANK, THE",5908 EAST MAIN STREET,FRIENDSHIP,IN,47021,41,0.001781,0.000013,0.004674,12,45.559483,0.535802,1,12198.0,NaN,NaN,NaN,NaN


In [ ]:
df[mask4]

,FDIC Certificate Number,Primary ABA Routing Number,Financial Institution Name,Financial Institution Address,Financial Institution City,Financial Institution State,Financial Institution Zip Code,Financial Institution Filing Type,Provisions,Asset Share,Failed Loans,Repite,Z_score,IL45,Disponible los 12 anios,Unnamed: 16,Latitude,Longitude,Latitude_GCP,Longitude_GCP
IDRSSD,,,,,,,,,,,,,,,,,,,,
427241,13130,74909988,"FRIENDSHIP STATE BANK, THE",5908 EAST MAIN STREET,FRIENDSHIP,IN,47021,41,0.001781,0.000013,0.004674,12,45.559483,0.535802,1,12198.0,NaN,NaN,NaN,NaN


In [ ]:
lat = 38.969938088695955
lon = -85.14857898008532

In [ ]:
df.loc[427241, ['Latitude_extra', 'Longitude_extra']] = lat, lon
df.loc[427241]

FDIC Certificate Number                                   13130
Primary ABA Routing Number                             74909988
Financial Institution Name           FRIENDSHIP STATE BANK, THE
Financial Institution Address            5908 EAST MAIN STREET 
Financial Institution City                           FRIENDSHIP
Financial Institution State                                  IN
Financial Institution Zip Code                            47021
Financial Institution Filing Type                            41
Provisions                                             0.001781
Asset Share                                            0.000013
Failed Loans                                           0.004674
Repite                                                       12
Z_score                                               45.559483
IL45                                                   0.535802
Disponible los 12 anios                                       1
Unnamed: 16                             

In [ ]:
df_mod_path = os.path.join(path, "Datos_Bancos_Lat_Lon.xlsx")
df_mod_path
# Guardar
# df.to_csv(df_mod_path)

'drive/MyDrive/Freelance/001/Datos_Bancos_Lat_Lon.xlsx'

In [ ]:
# Cargar
df_final = pd.read_csv(df_mod_path)
df_final.head()

,IDRSSD,FDIC Certificate Number,Primary ABA Routing Number,Financial Institution Name,Financial Institution Address,Financial Institution City,Financial Institution State,Financial Institution Zip Code,Financial Institution Filing Type,Provisions,...,Z_score,IL45,Disponible los 12 anios,Unnamed: 16,Latitude,Longitude,Latitude_GCP,Longitude_GCP,Latitude_extra,Longitude_extra
0,1397471,91280,82001687,BANK OF LITTLE ROCK,200 NORTH STATE STREET,LITTLE ROCK,AR,72201,41,0.002558,...,31.321814,0.585198,1,91280.0,34.750332,-92.278560,34.750552,-92.278918,NaN,NaN
1,455253,91005,307087713,5 STAR BANK (A COLORADO INDUSTRIAL BANK),"PETERSON AIRFORCE BASE, BLDG 1485",COLORADO SPRINGS,CO,80914,41,0.010991,...,23.623991,0.551245,0,90582.0,NaN,NaN,38.825656,-104.699599,NaN,NaN
2,896856,90582,307074124,CASTLE ROCK BANK,501 WILCOX STREET,CASTLE ROCK,CO,80104,41,0.000973,...,11.431871,0.420141,1,90528.0,39.376352,-104.859526,39.374558,-104.860375,NaN,NaN
3,522959,90528,302173732,MONTROSEBANK,200 NORTH TOWNSEND AVENUE,MONTROSE,CO,81401,41,0.001883,...,30.585695,0.578861,1,90384.0,38.514789,-107.917269,38.480826,-107.879260,NaN,NaN
4,734613,90384,31312123,"BANK OF LANDISBURG, THE",MAIN STREET,LANDISBURG,PA,17040,41,0.000000,...,93.394478,0.529710,1,90311.0,40.343500,-77.304215,40.343441,-77.304621,NaN,NaN


In [ ]:
gcp_cols = ['Latitude_GCP', 'Longitude_GCP']
mask_final = df_final[gcp_cols].isna().sum(axis=1) != 0
mask_final

0       False
1       False
2       False
3       False
4       False
        ...  
7725    False
7726    False
7727    False
7728    False
7729    False
Length: 7730, dtype: bool

In [ ]:
df_final.loc[mask_final]

,IDRSSD,FDIC Certificate Number,Primary ABA Routing Number,Financial Institution Name,Financial Institution Address,Financial Institution City,Financial Institution State,Financial Institution Zip Code,Financial Institution Filing Type,Provisions,...,Z_score,IL45,Disponible los 12 anios,Unnamed: 16,Latitude,Longitude,Latitude_GCP,Longitude_GCP,Latitude_extra,Longitude_extra
2944,427241,13130,74909988,"FRIENDSHIP STATE BANK, THE",5908 EAST MAIN STREET,FRIENDSHIP,IN,47021,41,0.001781,...,45.559483,0.535802,1,12198.0,NaN,NaN,NaN,NaN,38.969938,-85.148579


In [ ]:
extra_cols = ['Latitude_extra', 'Longitude_extra']
mask_extra = df_final[extra_cols].isna().sum(axis=1) == 0
mask_extra

0       False
1       False
2       False
3       False
4       False
        ...  
7725    False
7726    False
7727    False
7728    False
7729    False
Length: 7730, dtype: bool

In [ ]:
df_final[gcp_cols].loc[mask_final]

,Latitude_GCP,Longitude_GCP
2944,NaN,NaN


In [ ]:
df_final[gcp_cols].loc[mask_extra]

,Latitude_GCP,Longitude_GCP
2944,NaN,NaN


In [ ]:
df_final[extra_cols].loc[mask_final]

,Latitude_extra,Longitude_extra
2944,38.969938,-85.148579


In [ ]:
df_final[extra_cols].loc[mask_extra]

,Latitude_extra,Longitude_extra
2944,38.969938,-85.148579


In [ ]:
df_final.loc[mask_extra, gcp_cols]

,Latitude_GCP,Longitude_GCP
2944,NaN,NaN


In [ ]:
df_final.loc[2944, 'Latitude_GCP'] = df_final.loc[2944, 'Latitude_extra']
df_final.loc[2944, 'Latitude_GCP']

38.96993808869596

In [ ]:
df_final.loc[2944, 'Longitude_GCP'] = df_final.loc[2944, 'Longitude_extra']
df_final.loc[2944, 'Longitude_GCP']

-85.14857898008532

In [ ]:
cols_final = ['Latitude', 'Longitude']
df_final[cols_final]

,Latitude,Longitude
0,34.750332,-92.278560
1,NaN,NaN
2,39.376352,-104.859526
3,38.514789,-107.917269
4,40.343500,-77.304215
...,...,...
7725,41.018096,-92.413407
7726,28.296980,-97.275514
7727,37.334692,-79.524717
7728,40.807555,-91.143076


In [ ]:
df_final[gcp_cols]

,Latitude_GCP,Longitude_GCP
0,34.750552,-92.278918
1,38.825656,-104.699599
2,39.374558,-104.860375
3,38.480826,-107.879260
4,40.343441,-77.304621
...,...,...
7725,41.018125,-92.413345
7726,28.297959,-97.274970
7727,37.334570,-79.524733
7728,40.806718,-91.140924


In [ ]:
df_final[cols_final] = df_final[gcp_cols]
df_final[cols_final]

,Latitude,Longitude
0,34.750552,-92.278918
1,38.825656,-104.699599
2,39.374558,-104.860375
3,38.480826,-107.879260
4,40.343441,-77.304621
...,...,...
7725,41.018125,-92.413345
7726,28.297959,-97.274970
7727,37.334570,-79.524733
7728,40.806718,-91.140924


In [ ]:
df_final[cols_final].isna().sum()

Latitude     0
Longitude    0
dtype: int64

In [ ]:
gcp_cols + extra_cols

['Latitude_GCP', 'Longitude_GCP', 'Latitude_extra', 'Longitude_extra']

In [ ]:
df_final.drop(gcp_cols + extra_cols, axis=1, inplace=True)

In [ ]:
df_final[cols_final].isna().sum()

Latitude     0
Longitude    0
dtype: int64

In [ ]:
df_final.isna().sum()

IDRSSD                                  0
FDIC Certificate Number                 0
Primary ABA Routing Number              0
Financial Institution Name              0
Financial Institution Address           0
Financial Institution City              0
Financial Institution State             0
Financial Institution Zip Code          0
Financial Institution Filing Type       0
Provisions                              0
Asset Share                             0
Failed Loans                            0
Repite                                  0
Z_score                                 0
IL45                                    0
Disponible los 12 anios                 0
Unnamed: 16                          2489
Latitude                                0
Longitude                               0
dtype: int64

In [ ]:
df_final['(Lat, Lon)'] = '(' + df_final['Latitude'].astype(str) + ', ' + df_final['Longitude'].astype(str) + ')'
df_final['(Lat, Lon)']

0         (34.7505524, -92.278918)
1       (38.8256561, -104.6995991)
2        (39.3745579, -104.860375)
3       (38.4808256, -107.8792598)
4        (40.3434413, -77.3046208)
                   ...            
7725     (41.0181247, -92.4133449)
7726      (28.297959, -97.2749704)
7727     (37.3345703, -79.5247326)
7728     (40.8067183, -91.1409244)
7729     (35.0306461, -84.3853747)
Name: (Lat, Lon), Length: 7730, dtype: object

In [ ]:
df_final[['Latitude_comma', 'Longitude_comma']] = df_final[['Latitude', 'Longitude']]
df_final[['Latitude_comma', 'Longitude_comma']]

,Latitude_comma,Longitude_comma
0,34.750552,-92.278918
1,38.825656,-104.699599
2,39.374558,-104.860375
3,38.480826,-107.879260
4,40.343441,-77.304621
...,...,...
7725,41.018125,-92.413345
7726,28.297959,-97.274970
7727,37.334570,-79.524733
7728,40.806718,-91.140924


In [ ]:
df_final['Latitude_comma'] = df_final['Latitude_comma'].apply(formatear_numero)
df_final['Longitude_comma'] = df_final['Longitude_comma'].apply(formatear_numero)

In [ ]:
df_final[['Latitude_comma', 'Longitude_comma']]

,Latitude_comma,Longitude_comma
0,"34,7505524","-92,278918"
1,"38,8256561","-104,6995991"
2,"39,3745579","-104,860375"
3,"38,4808256","-107,8792598"
4,"40,3434413","-77,3046208"
...,...,...
7725,"41,0181247","-92,4133449"
7726,"28,297959","-97,2749704"
7727,"37,3345703","-79,5247326"
7728,"40,8067183","-91,1409244"


In [ ]:
df_final.to_excel(df_mod_path)

In [ ]:
# Crear un DataFrame de ejemplo
data = {'columna': [32.12345, 45.67890, 98.76543]}
df = pd.DataFrame(data)

# Función para formatear el número
def formatear_numero(x):
    return str(x).replace('.', ',')

# Aplicar la función a la columna
df['columna'] = df['columna'].apply(formatear_numero)

# Mostrar el DataFrame resultante
df.dtypes

columna    object
dtype: object